# DEFRA Dataset Assesment


1) I'll be start adding my main paths and modules I will be using in this notebook below.

In [30]:
# possible python modules i will be using below
from curses import meta
import os
import pandas as pd
from pathlib import Path
import csv

#define base path  without hardcoding
base_dir = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "defra" / "optimised"
#metadata file for pollutant name, location and site names
metadata_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "defra" /"test"/"std_london_sites_pollutant.csv"

# output path for saving statistics 1. function
#the first analyse dataset created without inclitiong nan optimased files, and cross referencing that's why changed the name to dataset_statistics-noNAN-incl.csv
os.makedirs(base_dir / "report", exist_ok=True)
stats_output_path = base_dir/"report"/ "defra_stats.csv"

# output paths for saving all the pollutant distribution and nan value analysis.
pollutant_distrubution_path = base_dir / "report" / "pollutant_distribution.csv"
nan_val_pollutant_split_path = base_dir / "report" / "nan_values_by_pollutant.csv"
nan_val_stationPollutant_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" /"defra" / "report" / "nan_values_by_station_pollutant.csv"


# log file from nan replacement process
nan_log_path = Path.home() / "Desktop" / "data science projects" / "air-pollution-levels" / "data" / "defra" / "logs" / "NaN_values_record.csv"

# possible python modules i will be using below


## 1) Initial Dataset Assessment: Raw Numbers

Before conducting quality checks, I need to establish the baseline characteristics of the DEFRA dataset. This section calculates comprehensive statistics about the data collection effort, including file counts, measurement records, station coverage, and pollutant distribution.

### Purpose
- Document the scale and scope of data collection.
- Establish baseline metrics for comparison with LAQN.
- Provide context for subsequent quality analysis.

### Methodology
The function `get_defra_dataset_statistics()` performs the following:
1. Loads standardised metadata to identify unique stations and pollutants.
2. Counts files across all three yearly directories (2023, 2024, 2025).
3. Calculates total measurement records by reading all CSV files.
4. Determines spatial coverage from unique coordinate pairs.
5. Documents temporal coverage (35 months: January 2023 to November 2025).

### Notes
- File counting is fast (scans directory structure only).
- Record counting can be slow (reads every CSV file).
- Results are saved to csv.

In [22]:
def get_defra_dataset_statistics(base_dir, metadata_path, nan_log_path):
    """
    Calculate statistics at DEFRA dataset.
    This function walks through the monthly data directories 2023, 2024, 2025 and calculates key metrics needed for reporting.
    
    Parameters:
        base_dir : Path
            Base directory containing defra data folders.
        metadata_path : Path
            Path to the standardised metadata csv file.
        nan_log_path : Path
            Path to the NaN values log file after notice data flags, changed them to NaN.
            
    Returns:
        dict : Dictionary containing all calculated statistics.
    """
    
    stats = {}
    
    # read metadata to get station and pollutant info
    print("\nloading metadata from std_london_sites_pollutant.csv...")
    metadata = pd.read_csv(metadata_path, encoding="utf-8")
    
    # calculate metadata statistics
    stats['unique_stations'] = metadata['station_name'].nunique()
    stats['total_combinations'] = len(metadata)
    stats['unique_pollutants'] = metadata['pollutant_std'].nunique()
    
    # get pollutant breakdown
    pollutant_counts = metadata['pollutant_std'].value_counts()
    stats['pollutant_distribution'] = pollutant_counts.to_dict()
    
    # create set of expected station/pollutant pairs from metadata
    expected_pairs = set(
        zip(metadata['station_name'], metadata['pollutant_std'])
    )
    stats['expected_pairs'] = len(expected_pairs)
    print(f"  expected station/pollutant pairs from metadata: {len(expected_pairs)}")
    
    # count unique coordinates for spatial coverage, i will be use this for laqn dataset as well
    # group by lat/lon and count unique locations, instead of station names and will do the validation afterwards
    unique_coords = metadata[['latitude', 'longitude']].drop_duplicates()
    stats['unique_locations'] = len(unique_coords)
    
    # count files in monthly data directories
    total_files = 0
    files_by_year = {}
    
    # loop through each years measurement directory
    print("\nscanning optimised directory for collected data...")
    for year in ['2023', '2024', '2025']:
        year_dir = Path(base_dir) / f'{year}measurements'
        
        if year_dir.exists():
            # count all CSV files in this years directory and subdirectories
            year_files = list(year_dir.rglob('*.csv'))
            files_by_year[year] = len(year_files)
            total_files += len(year_files)
            print(f"  {year}: {len(year_files)} files")
        else:
            files_by_year[year] = 0
            print(f"  {year}: directory not found")
    
    stats['total_files'] = total_files
    stats['files_by_year'] = files_by_year
    
    # calculate total measurement records, this requires reading all csv files and counting rows
    total_records = 0
    records_by_year = {}
    total_missing = 0
    missing_by_year = {}
    
    # concatenate all CSVs for missing value breakdown
    all_csvs = []
    
    print("\nreading all CSV files to calculate statistics...")
    for year in ['2023', '2024', '2025']:
        year_dir = Path(base_dir) / f'{year}measurements'
        year_records = 0
        year_missing = 0
        
        if year_dir.exists():
            # read each csv, count rows and missing values
            for csv_file in year_dir.rglob('*.csv'):
                try:
                    df = pd.read_csv(csv_file)
                    year_records += len(df)
                    
                    # count missing NaN or empty string values in value column
                    # calculation: missing values in value column only
                    if 'value' in df.columns:
                        missing_in_file = df['value'].isna().sum() + (df['value'] == "").sum()
                        year_missing += missing_in_file
                    
                    # store dataframe for later aggregation
                    all_csvs.append(df)
                    
                except Exception as e:
                    print(f"  warning: could not read {csv_file.name}: {e}")
            
            records_by_year[year] = year_records
            missing_by_year[year] = year_missing
            total_records += year_records
            total_missing += year_missing
            print(f"  {year}: {year_records:,} records, {year_missing:,} missing ({(year_missing/year_records*100):.2f}%)")
        else:
            records_by_year[year] = 0
            missing_by_year[year] = 0
    
    stats['total_records'] = total_records
    stats['records_by_year'] = records_by_year
    stats['missing_by_year'] = missing_by_year
    stats['total_missing'] = total_missing
    stats['overall_completeness'] = ((total_records - total_missing) / total_records * 100) if total_records > 0 else 0
    
    # cross-reference metadata with collected data
    print("\ncross-referencing collected data with metadata...")
    
    if all_csvs:
        all_data = pd.concat(all_csvs, ignore_index=True)
        
        # check if required columns exist in csv files
        # file structure: timestamp,value,timeseries_id,station_name,pollutant_name,pollutant_std,latitude,longitude
        if 'station_name' in all_data.columns and 'pollutant_std' in all_data.columns:
            # identify actual station/pollutant pairs in collected data
            collected_pairs = set(
                zip(all_data['station_name'], all_data['pollutant_std'])
            )
            stats['collected_pairs'] = len(collected_pairs)
            
            # find missing pairs (in metadata but not in collected data)
            missing_pairs = expected_pairs - collected_pairs
            stats['missing_pairs'] = list(missing_pairs)
            stats['missing_pairs_count'] = len(missing_pairs)
            
            # find extra pairs (in collected data but not in metadata)
            extra_pairs = collected_pairs - expected_pairs
            stats['extra_pairs'] = list(extra_pairs)
            stats['extra_pairs_count'] = len(extra_pairs)
            
            print(f"  expected pairs from metadata: {len(expected_pairs)}")
            print(f"  actually collected pairs: {len(collected_pairs)}")
            print(f"  missing pairs (in metadata but not collected): {len(missing_pairs)}")
            print(f"  extra pairs (collected but not in metadata): {len(extra_pairs)}")
            
            # group by station and pollutant_std, count missing values
            # calculation: (100 * missing value cell number) / (total number of row value col)
            missing_breakdown = {}
            
            for (station, pollutant), group in all_data.groupby(['station_name', 'pollutant_std']):
                total_rows = len(group)
                # count missing in value column
                if 'value' in group.columns:
                    missing_rows = group['value'].isna().sum() + (group['value'] == "").sum()
                else:
                    missing_rows = 0
                
                missing_breakdown[(station, pollutant)] = (int(missing_rows), int(total_rows))
            
            stats['missing_by_station_pollutant'] = missing_breakdown
        else:
            print("  warning: station_name or pollutant_std columns not found")
            stats['missing_by_station_pollutant'] = {}
            stats['collected_pairs'] = 0
            stats['missing_pairs'] = []
            stats['missing_pairs_count'] = 0
            stats['extra_pairs'] = []
            stats['extra_pairs_count'] = 0
    else:
        stats['missing_by_station_pollutant'] = {}
        stats['collected_pairs'] = 0
        stats['missing_pairs'] = list(expected_pairs)
        stats['missing_pairs_count'] = len(expected_pairs)
        stats['extra_pairs'] = []
        stats['extra_pairs_count'] = 0
    
    # distribution of nan by pollutant over time
    if stats['missing_by_station_pollutant']:
        pollutant_missing_summary = {}
        
        for (station, pollutant), (missing, total) in stats['missing_by_station_pollutant'].items():
            if pollutant not in pollutant_missing_summary:
                pollutant_missing_summary[pollutant] = {'total_missing': 0, 'total_records': 0}
            
            pollutant_missing_summary[pollutant]['total_missing'] += missing
            pollutant_missing_summary[pollutant]['total_records'] += total
        
        # calculate percentages
        for pollutant in pollutant_missing_summary:
            total_missing = pollutant_missing_summary[pollutant]['total_missing']
            total_records = pollutant_missing_summary[pollutant]['total_records']
            percentage = (total_missing / total_records * 100) if total_records > 0 else 0
            pollutant_missing_summary[pollutant]['percentage_missing'] = percentage
        
        stats['missing_by_pollutant_type'] = pollutant_missing_summary
    else:
        stats['missing_by_pollutant_type'] = {}
    
    # log file created during data cleaning process
    if Path(nan_log_path).exists():
        nan_log = pd.read_csv(nan_log_path)
        
        # calculate replacement statistics per year
        replacements_by_year = nan_log.groupby('year_folder')['invalid_flags_replaced'].sum().to_dict()
        stats['nan_replacements_by_year'] = replacements_by_year
        stats['total_nan_replacements'] = nan_log['invalid_flags_replaced'].sum()
        
        # get mean percentage of invalid flags
        stats['mean_invalid_percentage'] = nan_log['percentage_invalid'].mean()
        stats['max_invalid_percentage'] = nan_log['percentage_invalid'].max()
        
    else:
        stats['nan_replacements_by_year'] = {}
        stats['total_nan_replacements'] = 0
        stats['mean_invalid_percentage'] = 0
        stats['max_invalid_percentage'] = 0
    
    # calculate temporal coverage based on the files collected, understands which months have data
    stats['temporal_coverage'] = {
        'start_date': '2023-01-01',
        'end_date': '2025-11-19',  
        'total_months': 35
    }
    
    return stats

In [27]:
def print_dataset_statistics(stats):
    """
    Print dataset statistics
    
    Parameters:
        stats : dict
            returned by get_defra_dataset_statistics().
    """
    
    print("\n" + "="*40)
    print("Defra dataset statistics: initial assessment")
    print("="*40)
    
    print("\nScale and scope:")
    print(f"Total files collected: {stats['total_files']:,}")
    print(f"Total measurement records: {stats['total_records']:,}")
    print(f"Total missing values (nan): {stats['total_missing']:,}")
    print(f"Overall completeness: {stats['overall_completeness']:.2f}%")
    print(f"Unique monitoring stations: {stats['unique_stations']}")
    print(f"Total station-pollutant combinations: {stats['total_combinations']}")
    print(f"Unique pollutant types: {stats['unique_pollutants']}")
    print(f"Unique geographic locations: {stats['unique_locations']}")
    
    # data collection coverage
    print("\nData collection coverage:")
    print(f"Expected pairs (from metadata): {stats.get('expected_pairs', 0)}")
    print(f"Actually collected pairs: {stats.get('collected_pairs', 0)}")
    print(f"Missing pairs (not collected): {stats.get('missing_pairs_count', 0)}")
    print(f"Extra pairs (not in metadata): {stats.get('extra_pairs_count', 0)}")
    
    if stats.get('missing_pairs_count', 0) > 0:
        print(f"\nwarning: {stats['missing_pairs_count']} station/pollutant pairs from metadata were not found in collected data.")
        print("first 10 missing pairs:")
        for i, (station, pollutant) in enumerate(stats['missing_pairs'][:10], 1):
            print(f"  {i}. {station} - {pollutant}")
    
    if stats.get('extra_pairs_count', 0) > 0:
        print(f"\nNote: {stats['extra_pairs_count']} station/pollutant pairs in collected data are not in metadata.")
    
    print("\nfiles by year:")
    for year, count in stats['files_by_year'].items():
        print(f"  {year}: {count:,} files")
    
    print("\nrecords by year:")
    for year, count in stats['records_by_year'].items():
        missing = stats['missing_by_year'].get(year, 0)
        missing_pct = (missing / count * 100) if count > 0 else 0
        print(f"  {year}: {count:,} records, {missing:,} missing ({missing_pct:.2f}%)")
    
    # adding nan value summary below
    print("\nnan replacement summary:")
    print(f"Total invalid flags replaced: {stats['total_nan_replacements']:,}")
    print(f"Mean invalid percentage per file: {stats['mean_invalid_percentage']:.2f}%")
    print(f"Max invalid percentage: {stats['max_invalid_percentage']:.2f}%")
    
    # count of replacements by year
    if stats['nan_replacements_by_year']:
        print("\nreplacements by year:")
        for year_folder, count in stats['nan_replacements_by_year'].items():
            print(f"  {year_folder}: {count:,} flags replaced")
    
    print("\ntemporal coverage:")
    print(f"start date: {stats['temporal_coverage']['start_date']}")
    print(f"end date: {stats['temporal_coverage']['end_date']}")
    print(f"total months: {stats['temporal_coverage']['total_months']}")
    
    print("\npollutant distribution:")
    print("station/pollutant combinations by type:")
    for pollutant, count in sorted(stats['pollutant_distribution'].items(), 
                                   key=lambda x: x[1], reverse=True):
        percentage = (count / stats['total_combinations']) * 100
        print(f"  {pollutant}: {count} ({percentage:.1f}%)")
    
    # missing value distribution by pollutant type
    print("\nMissing value distribution by pollutant type:")
    if stats.get('missing_by_pollutant_type'):
        # sort by percentage missing (highest first)
        sorted_pollutants = sorted(
            stats['missing_by_pollutant_type'].items(),
            key=lambda x: x[1]['percentage_missing'],
            reverse=True
        )
        
        print(f"{'pollutant':<20} {'total records':>15} {'missing':>12} {'% missing':>12}")
        print("-" * 60)
        for pollutant, data in sorted_pollutants:
            print(f"{pollutant:<20} {data['total_records']:>15,} {data['total_missing']:>12,} {data['percentage_missing']:>11.2f}%")
    else:
        print("  no missing value distribution available.")
    
    # print missing values by station/pollutant breakdown with row_number column
    print("\nMissing values by station/pollutant:")
    if stats.get('missing_by_station_pollutant'):
        # prepare a sorted list by missing percentage descending
        breakdown = []
        for (station, pollutant), (missing, total) in stats['missing_by_station_pollutant'].items():
            percent = (missing / total * 100) if total > 0 else 0
            breakdown.append((station, pollutant, missing, total, percent))
        # sort by percentage descending and take top 20
        breakdown.sort(key=lambda x: x[4], reverse=True)
        breakdown = breakdown[:20]
        print(f"{'station':<30} {'pollutant':<20} {'missing':>10} {'total_row':>12} {'% missing':>12}")
        print("-" * 40)
        
        for station, pollutant, missing, total, percent in breakdown:
            print(f"{station:<30} {pollutant:<20} {missing:>10,} {total:>12,} {percent:>11.2f}%")
    else:
        print(" No missing value breakdown available.")

In [ ]:
# run the analysis
stats = get_defra_dataset_statistics(base_dir, metadata_path, nan_log_path)
print_dataset_statistics(stats)

# # Save statistics for later use as csv
# save statistics for later use as csv
# prepare flat data structure for csv
stats_rows = []
stats_rows.append(["metric", "value"])
stats_rows.append(["total_files", stats['total_files']])
stats_rows.append(["total_records", stats['total_records']])
stats_rows.append(["total_missing", stats['total_missing']])
stats_rows.append(["overall_completeness_pct", f"{stats['overall_completeness']:.2f}"])
stats_rows.append(["unique_stations", stats['unique_stations']])
stats_rows.append(["total_combinations", stats['total_combinations']])
stats_rows.append(["unique_pollutants", stats['unique_pollutants']])
stats_rows.append(["unique_locations", stats['unique_locations']])
stats_rows.append(["expected_pairs", stats.get('expected_pairs', 0)])
stats_rows.append(["collected_pairs", stats.get('collected_pairs', 0)])
stats_rows.append(["missing_pairs_count", stats.get('missing_pairs_count', 0)])
stats_rows.append(["extra_pairs_count", stats.get('extra_pairs_count', 0)])
stats_rows.append(["total_nan_replacements", stats['total_nan_replacements']])
stats_rows.append(["mean_invalid_pct", f"{stats['mean_invalid_percentage']:.2f}"])
stats_rows.append(["max_invalid_pct", f"{stats['max_invalid_percentage']:.2f}"])

# add year-specific metrics
for year in ['2023', '2024', '2025']:
    stats_rows.append([f"files_{year}", stats['files_by_year'].get(year, 0)])
    stats_rows.append([f"records_{year}", stats['records_by_year'].get(year, 0)])
    stats_rows.append([f"missing_{year}", stats['missing_by_year'].get(year, 0)])
    year_key = f'{year}measurements'
    stats_rows.append([f"replacements_{year}", stats['nan_replacements_by_year'].get(year_key, 0)])

# # save to csv stats report save func below (commented out for now to overwrite previous report)
# pd.DataFrame(stats_rows[1:], columns=stats_rows[0]).to_csv(stats_output_path, index=False)
# print(f"\nstatistics saved to: {stats_output_path}")

# # save pollutant distribution to csv describe the path on top pollutant_distrubution_path
# total_combinations = stats['total_combinations']
# pollutant_distribution_df = pd.DataFrame(
#     [
#         {
#             'pollutant': k,
#             'count': v,
#             'percentage': round((v / total_combinations) * 100, 2) if total_combinations > 0 else 0
#         }
#         for k, v in stats['pollutant_distribution'].items()
#     ]
# )
# pollutant_distribution_df.to_csv(pollutant_distrubution_path, index=False)
# print(f"Pollutant distribution saved to: {pollutant_distrubution_path}")

# # Save missing value distribution by pollutant type to path described the path on top nan_val_pollutant_split_path
# if stats.get('missing_by_pollutant_type'):
#     missing_by_pollutant_df = pd.DataFrame([
#         {
#             'pollutant': k,
#             'total_records': v['total_records'],
#             'total_missing': v['total_missing'],
#             'percentage_missing': v['percentage_missing']
#         }
#         for k, v in stats['missing_by_pollutant_type'].items()
#     ])
#     missing_by_pollutant_df.to_csv(nan_val_pollutant_split_path, index=False)
#     print(f"Missing value distribution by pollutant type saved to: {nan_val_pollutant_split_path}")

# # save missing values by station/pollutant to csv path on top nan_val_stationPollutant_path
# if stats.get('missing_by_station_pollutant'):
#     missing_by_station_pollutant_df = pd.DataFrame([
#         {
#             'station': k[0],
#             'pollutant': k[1],
#             'missing': v[0],
#             'total_row': v[1],
#             'percentage_missing': (v[0] / v[1] * 100) if v[1] > 0 else 0
#         }
#         for k, v in stats['missing_by_station_pollutant'].items()
#     ])
#     missing_by_station_pollutant_df.to_csv(nan_val_stationPollutant_path, index=False)
#     print(f"Missing values by station/pollutant saved to: {nan_val_stationPollutant_path}")



loading metadata from std_london_sites_pollutant.csv...
  expected station/pollutant pairs from metadata: 141

scanning optimised directory for collected data...
  2023: 1431 files
  2024: 1193 files
  2025: 939 files

reading all CSV files to calculate statistics...
  2023: 1,000,126 records, 90,161 missing (9.01%)
  2024: 868,320 records, 101,256 missing (11.66%)
  2025: 657,545 records, 30,750 missing (4.68%)

cross-referencing collected data with metadata...
  expected pairs from metadata: 141
  actually collected pairs: 141
  missing pairs (in metadata but not collected): 0
  extra pairs (collected but not in metadata): 0

Defra dataset statistics: initial assessment

Scale and scope:
Total files collected: 3,563
Total measurement records: 2,525,991
Total missing values (nan): 222,167
Overall completeness: 91.20%
Unique monitoring stations: 18
Total station-pollutant combinations: 144
Unique pollutant types: 37
Unique geographic locations: 20

Data collection coverage:
Expected p

    loading metadata from std_london_sites_pollutant.csv...
    expected station/pollutant pairs from metadata: 141

    scanning optimised directory for collected data...
    2023: 1431 files
    2024: 1193 files
    2025: 939 files

    reading all CSV files to calculate statistics...
    2023: 1,000,126 records, 90,161 missing (9.01%)
    2024: 868,320 records, 101,256 missing (11.66%)
    2025: 657,545 records, 30,750 missing (4.68%)

    cross-referencing collected data with metadata...
    expected pairs from metadata: 141
    actually collected pairs: 141
    missing pairs (in metadata but not collected): 0
    extra pairs (collected but not in metadata): 0

    ========================================
    Defra dataset statistics: initial assessment
    ========================================

    Scale and scope:
    Total files collected: 3,563
    Total measurement records: 2,525,991
    Total missing values (nan): 222,167
    Overall completeness: 91.20%
    Unique monitoring stations: 18
    Total station-pollutant combinations: 144
    Unique pollutant types: 37
    Unique geographic locations: 20

    Data collection coverage:
    Expected pairs (from metadata): 141
    Actually collected pairs: 141
    Missing pairs (not collected): 0
    Extra pairs (not in metadata): 0

    files by year:
    2023: 1,431 files
    2024: 1,193 files
    2025: 939 files

    records by year:
    2023: 1,000,126 records, 90,161 missing (9.01%)
    2024: 868,320 records, 101,256 missing (11.66%)
    2025: 657,545 records, 30,750 missing (4.68%)

    nan replacement summary:
    Total invalid flags replaced: 222,167
    Mean invalid percentage per file: 9.61%
    Max invalid percentage: 100.00%

    replacements by year:
    2023measurements: 90,161 flags replaced
    2024measurements: 101,256 flags replaced
    2025measurements: 30,750 flags replaced

    temporal coverage:
    start date: 2023-01-01
    end date: 2025-11-19
    total months: 35

    pollutant distribution:
    station/pollutant combinations by type:
    PM10: 15 (10.4%)
    PM2.5: 15 (10.4%)
    NO2: 14 (9.7%)
    NOx: 14 (9.7%)
    NO: 14 (9.7%)
    O3: 9 (6.2%)
    SO2: 3 (2.1%)
    n-Pentane: 2 (1.4%)
    m,p-Xylene: 2 (1.4%)
    n-Butane: 2 (1.4%)
    n-Heptane: 2 (1.4%)
    n-Hexane: 2 (1.4%)
    n-Octane: 2 (1.4%)
    Propene: 2 (1.4%)
    o-Xylene: 2 (1.4%)
    Propane: 2 (1.4%)
    i-Pentane: 2 (1.4%)
    Toluene: 2 (1.4%)
    trans-2-Butene: 2 (1.4%)
    trans-2-Pentene: 2 (1.4%)
    Isoprene: 2 (1.4%)
    Ethyne: 2 (1.4%)
    i-Octane: 2 (1.4%)
    i-Hexane: 2 (1.4%)
    i-Butane: 2 (1.4%)
    Ethylbenzene: 2 (1.4%)
    Ethene: 2 (1.4%)
    Ethane: 2 (1.4%)
    cis-2-Butene: 2 (1.4%)
    Benzene: 2 (1.4%)
    1-Pentene: 2 (1.4%)
    1-Butene: 2 (1.4%)
    1,3-Butadiene: 2 (1.4%)
    1,3,5-TMB: 2 (1.4%)
    1,2,4-TMB: 2 (1.4%)
    1,2,3-TMB: 2 (1.4%)
    CO: 2 (1.4%)

    Missing value distribution by pollutant type:
    pollutant              total records      missing    % missing
    ------------------------------------------------------------
    PM10                         227,142       37,580       16.54%
    O3                           194,333       27,184       13.99%
    PM2.5                        234,748       29,623       12.62%
    SO2                           72,928        7,181        9.85%
    NO                           326,061       25,444        7.80%
    NO2                          326,072       25,429        7.80%
    NOx                          325,387       24,964        7.67%
    n-Octane                      26,649        1,764        6.62%
    CO                            48,578        3,078        6.34%
    m,p-Xylene                    25,503        1,612        6.32%
    1,3,5-TMB                     26,649        1,641        6.16%
    Toluene                       26,649        1,640        6.15%
    i-Octane                      26,649        1,624        6.09%
    n-Heptane                     26,649        1,622        6.09%
    1,2,4-TMB                     26,649        1,610        6.04%
    Ethylbenzene                  26,649        1,592        5.97%
    Benzene                       26,649        1,586        5.95%
    o-Xylene                      26,649        1,568        5.88%
    1,2,3-TMB                     26,649        1,560        5.85%
    1-Pentene                     26,572        1,381        5.20%
    cis-2-Butene                  26,599        1,378        5.18%
    trans-2-Pentene               26,599        1,366        5.14%
    Isoprene                      26,618        1,341        5.04%
    Ethyne                        26,529        1,328        5.01%
    1,3-Butadiene                 26,568        1,320        4.97%
    i-Hexane                      26,599        1,321        4.97%
    trans-2-Butene                26,599        1,321        4.97%
    n-Hexane                      26,580        1,320        4.97%
    Propane                       26,618        1,316        4.94%
    Ethane                        26,599        1,315        4.94%
    Ethene                        26,618        1,312        4.93%
    Propene                       26,618        1,312        4.93%
    i-Butane                      26,599        1,308        4.92%
    1-Butene                      26,599        1,307        4.91%
    n-Butane                      26,599        1,307        4.91%
    i-Pentane                     26,618        1,306        4.91%
    n-Pentane                     26,618        1,306        4.91%

    Missing values by station/pollutant:
    station                        pollutant               missing    total_row    % missing
    ----------------------------------------
    London Eltham                  PM10                     16,337       16,826       97.09%
    London Eltham                  NO2                      13,187       16,840       78.31%
    London Eltham                  NO                       13,182       16,835       78.30%
    London Eltham                  NOx                      13,125       16,793       78.16%
    London Eltham                  O3                       12,537       16,842       74.44%
    London Teddington Bushy Park   PM10                     10,525       24,327       43.26%
    London Teddington Bushy Park   PM2.5                    20,820       48,656       42.79%
    London Haringey Priory Park South O3                        8,171       24,288       33.64%
    London Marylebone Road         PM10                        632        2,355       26.84%
    London Marylebone Road         PM2.5                       479        2,355       20.34%
    London Norbury Manor School    PM10                        936        5,258       17.80%
    London Norbury Manor School    PM2.5                       936        5,258       17.80%
    London Bexley                  PM10                      4,012       24,273       16.53%
    Southwark A2 Old Kent Road     PM10                        388        2,355       16.48%
    Haringey Roadside              NOx                       3,725       24,250       15.36%
    Haringey Roadside              NO2                       3,708       24,285       15.27%
    Haringey Roadside              NO                        3,708       24,287       15.27%
    London Westminster             PM2.5                     3,463       24,299       14.25%
    London Marylebone Road         SO2                       2,987       24,290       12.30%
    London Marylebone Road         CO                        2,729       24,293       11.23%
    Pollutant distribution saved to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/defra/optimised/report/pollutant_distribution.csv

## 2) Spatial Coverage Analysis

 analysing spatial distribution patterns before accepting the dataset. I need to understand where defra stations are located, identify any geographic biases, and compare coverage to laqn.

### Purpose
- Create maps showing station locations across London.
- Analyse density by borough to identify coverage gaps
- Compare spatial distribution to laqn network
- Ensure no geographic areas are overrepresented or underrepresented

### Methodology
1. Load defra metadata with coordinates
2. Create interactive folium map showing all stations
3. Calculate station density by borough
4. Identify coverage gaps in london
5. Compare to laqn spatial distribution



sources: 
- https://python-visualization.github.io/folium/latest/getting_started.html
- https://pandas.pydata.org/docs/user_guide/groupby.html 
- plotting: https://geopandas.org/en/stable/docs/user_guide/data_structures.html#geoseries
    - general: https://geopandas.org/en/stable/getting_started.html

coordinates:
  -  https://www.ordnancesurvey.co.uk/
  - identifiers: https://www.ordnancesurvey.co.uk/products/search-for-os-products?category=387aa470-8f46-4b02-a4ea-b70d1835f812 
  - WGS84 coordinate system used for latitude/longitude.
  - london coordinates : 51.5072° N, 0.1276° W
  - Latitude and longitude coordinates are: 51.509865, -0.118092.

In [ ]:
def analyse_spatial_coverage(metadata_path):
    """
    analyse the stations location on map 
    
    function validates coordinates, identifies  locations, and visulise the spatial distribution
    
    Parameters:
        metadata_path : 
             std metadata csv file.
            
    Returns:
        dictionary containing spatial statistics and coordinate data.
    
        *i got help for this section, sources folium tuttorials, plotting for geopandas and google. Also asked for my friend help as well which
        she works on geospatial data a lot for her phd research.

    """
    
    spatial_stats = {}
    
    # read metadata for coordinate information
    print("\nloading station coordinates from metadata...")
    metadata = pd.read_csv(metadata_path, encoding="utf-8")
    
    # check if coordinate columns exist
    if 'latitude' not in metadata.columns or 'longitude' not in metadata.columns:
        print("  error: latitude or longitude columns not found in metadata")
        return spatial_stats
    
    # validate coordinate completeness
    total_stations = len(metadata)
    missing_lat = metadata['latitude'].isna().sum()
    missing_lon = metadata['longitude'].isna().sum()
    missing_coords = metadata[['latitude', 'longitude']].isna().any(axis=1).sum()
    
    spatial_stats['total_stations'] = total_stations
    spatial_stats['missing_coordinates'] = missing_coords
    spatial_stats['missing_latitude'] = missing_lat
    spatial_stats['missing_longitude'] = missing_lon
    spatial_stats['coordinate_completeness'] = ((total_stations - missing_coords) / total_stations * 100) if total_stations > 0 else 0
    
    print(f"  total stations in metadata: {total_stations}")
    print(f"  missing coordinates: {missing_coords} ({(missing_coords/total_stations*100):.2f}%)")
    print(f"  coordinate completeness: {spatial_stats['coordinate_completeness']:.2f}%")
    
   

In [ ]:
def print_spatial_statistics(spatial_stats):
    """
    Print spatial coverage statistics 
    
    Param:
        spatial_stats : 
            Dic returned by analyse_spatial_coverage()
    """
    
    

## 3) Data Quality validations:


A critical gap from the laqn report by applying formal statistical tests to validate data quality patterns. While descriptive statistics show 0% (before I notice the flags of the dataset) issue rate, I need statistical evidence that this pattern is real and not due to chance.


#### Purpuse:
 Checking data qualities if it is in the limits of eea, and make sence for general logic.
- Outlier detection in pollutant measurements.
- Data validity ranges based on WHO/EEA standards.
- Measurement consistency across time periods.
- Quality flags and suspicious patterns.

### methodology
 applies environmental data quality assessment standards:
1. Load aggregated measurement data from all csv files.
2. Calculate statistical distributions for each pollutant type.
3. Identify outliers using IQR method and domain knowledge.
4. Check values against established valid ranges.
5. Flag suspicious patterns constant values, extreme spikes.
6. Calculate quality scores for each station-pollutant combination.

#### air quality measurement standards

- Uk air quality objectives, limits and policy.
- https://uk-air.defra.gov.uk/air-pollution/uk-limits
- chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://uk-air.defra.gov.uk/assets/documents/Air_Quality_Objectives_Update_20230403.pdf

- DEFRA. (2023). *Air Pollution in the UK 2022*.
  - Source: https://uk-air.defra.gov.uk/library/annualreport/
  - Air Quality Objectives and limit values
  - Compliance assessment methodology

- UK Air Information Resource. (2024). *Air Pollution: UK Limits*.
  - Source: https://uk-air.defra.gov.uk/air-pollution/uk-limits
  - Current UK air quality objectives
  - Legal limit values and target dates
  - Measurement unit specifications (µg/m³)
